# **2021ML FALL HW1: PM2.5 Prediction (Regression)**

# **Import Some Packages**

In [1]:
# For data preprocess
import numpy as np
import csv
import math
import pandas as pd
import math

In [ ]:
data_train = pd.read_csv('../train.csv')
data_test = pd.read_csv('../test.csv')

# Data preprocessing

In [ ]:
class DataPreprocessor():
    def __init__(self):
        self.keep_col = None
        self.top_corr_number = 15
        self.train_mean = None
        self.train_std = None
    
    def remove_outlier_y_with_IQR(self, data_train: pd.DataFrame):
        mean_y = data_train['PM2.5'].mean()
        q75, q25 = np.percentile(data_train['PM2.5'], [75 ,25])
        IQR = q75 - q25
        upper_bound = mean_y + 1.5 * IQR
        lower_bound = mean_y - 1.5 * IQR
        keep_cond = (data_train['PM2.5'] < upper_bound) & (data_train['PM2.5'] > lower_bound)
        data_train = data_train[keep_cond]
        return data_train

    def remove_larger_than_1000(self, data_train: pd.DataFrame):
        data_train = data_train[data_train['PM2.5'] < 1000]
        return data_train

    def slice_top_n_correlation_with_y(self, data: pd.DataFrame, isTraining = False):
        if isTraining:
            index_corr_descending = abs(data.corrwith(data['PM2.5'])).sort_values(ascending=False).index
            #first column will always be `PM2.5`
            self.keep_col = index_corr_descending[:self.top_corr_number]
        data = data[self.keep_col]
        return data

    def normalize_data(self, X_data: np.array, isTraining = False):
        if isTraining:
            self.train_mean = X_data.mean(axis = 0)
            self.train_std = X_data.std(axis = 0)
        normalized_data = (X_data - self.train_mean) / self.train_std
        return normalized_data

    def min_max_scaling(self, X_data: np.array, isTraining = False):
        if isTraining:
            self.train_max = X_data.max(axis = 0)
            self.train_min = X_data.min(axis = 0)
        normalized_data = (X_data - self.train_min) / (self.train_max - self.train_min)
        return normalized_data


    def parse2train(self, data):
        data = data.values
        x = []
        y = []
        # 用前面9筆資料預測下一筆PM2.5 所以需要-9
        total_length = data.shape[0] - 9
        for i in range(total_length):
            x_tmp = data[i:i+9, :]
            y_tmp = data[i+9, 0] #PM2.5 is the first column
            x.append(x_tmp.reshape(-1,))
            y.append(y_tmp)
        x = np.array(x) #shape = (# of observations, # of features * # of time lag)
        y = np.array(y)
        return x, y


    def parse2test(self, data):
        x = []
        y = []
        data = data.values
        # 用前面9筆資料預測下一筆PM2.5 所以需要-9
        total_length = data.shape[0] - 9
        for i in range(857):
            x_tmp = data[9 * i: 9 * i + 9, :]
            x.append(x_tmp.reshape(-1,))
        # x 會是一個(n, 18, 9)的陣列， y 則是(n, 1) 
        x = np.array(x)
        return x

    def preprocess_train_data(self, data_train: pd.DataFrame):
        data_train = self.remove_outlier_y_with_IQR(data_train)
        #data_train = self.remove_larger_than_1000(data_train)
        data_train = self.slice_top_n_correlation_with_y(data_train, isTraining=True)
        X_train, y_train = self.parse2train(data_train)
        #X_train = self.normalize_data(X_train, isTraining = True)
        X_train = self.min_max_scaling(X_train, isTraining = True)
        return X_train, y_train

    def preprocess_test_data(self, data_test: pd.DataFrame):
        data_test = self.slice_top_n_correlation_with_y(data_test, isTraining=False)
        X_test = self.parse2test(data_test)
        #X_test = self.normalize_data(X_test, isTraining = False)
        X_test = self.min_max_scaling(X_test, isTraining = False)
        return X_test

In [ ]:
class LinearRegression():
    def __init__(self):
        self.train_valid_ratio = 0.70
        self.train_loss = list()
        self.valid_loss = list()
        #record best information
        self.best_w = None
        self.best_bias = None
        self.best_valid_RMSE = None
        self.best_epoch = None
        self.stop_epoch = None

    def initialize_params(self, x):
        #initalize as a ~ unif(1, n-1) / sqrt(n)
        w = np.random.rand(x.shape[1]).reshape(-1, 1)
        bias = np.random.rand()
        return w, bias

    def train(self, x, y,batch_size, epoch_size, learning_rate, lam_train, verbose = True):
        #Initialize parameter
        batch_size = batch_size 
        epoch_size = epoch_size 
        lr = learning_rate
        w, bias = self.initialize_params(x)
        best_valid_RMSE = 99999
        best_epoch = 0
        patience = 10 #for early stopping

        #Adam optimizer
        lam = lam_train #L2 regularization
        beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1) #exponential decay rates for the moment estimates
        beta_2 = np.full(x[0].shape, 0.999).reshape(-1,1) #exponential decay rates for the moment estimates
        #m_t: first momentum vector, for momentum
        m_t = np.full(x[0].shape, 0).reshape(-1, 1)
        #v_t: second momemtum vector, for RMSprop
        v_t = np.full(x[0].shape, 0).reshape(-1, 1)
        m_t_b = 0.0
        v_t_b = 0.0
        t = 0
        epsilon = 1e-8

        for num_epoch in range(1, epoch_size + 1):
            #Shuffle when each epoch begin
            index = np.arange(x.shape[0])
            np.random.shuffle(index)
            x = x[index]
            y = y[index]
            split_point_x = math.floor(x.shape[0] * self.train_valid_ratio)
            split_point_y = math.floor(y.shape[0] * self.train_valid_ratio)
            X_train = x[:split_point_x, :]
            y_train = y[:split_point_y] 
            X_valid = x[split_point_x:, :] 
            y_valid = y[split_point_y:]
            SSE = 0
            for num_batch in range(int(X_train.shape[0]/batch_size)):
                t+=1
                x_batch = X_train[num_batch * batch_size:(num_batch + 1) * batch_size]
                y_batch = y_train[num_batch * batch_size:(num_batch + 1) * batch_size].reshape(-1,1)
                
                y_hat = np.dot(x_batch, w) + bias
                train_loss = y_batch - y_hat
                # 計算gradient
                g_t = np.dot(x_batch.transpose(), train_loss) * (-2) +  2 * lam * np.sum(w)
                #助教原本寫的是*2
                g_t_b = train_loss.sum(axis=0) * (-2)
                m_t = beta_1*m_t + (1-beta_1)*g_t 
                v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
                m_cap = m_t/(1-(beta_1**t))
                v_cap = v_t/(1-(beta_2**t))
                m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
                v_t_b = 0.999*v_t_b + (1-0.999) * (g_t_b*g_t_b) 
                m_cap_b = m_t_b/(1-(0.9**t))
                v_cap_b = v_t_b/(1-(0.999**t))

                # 更新weight, bias
                w -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
                bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)
                SSE += np.sum(np.square(train_loss))

            # print(SSE)
            train_RMSE = np.sqrt(SSE / X_train.shape[0])
            valid_RMSE = self.compute_valid_RMSE(X_valid, y_valid, w, bias)
            self.train_loss.append(train_RMSE)
            self.valid_loss.append(valid_RMSE)
            if verbose:
                print(f"Epoch {num_epoch}, training RMSE = {round(train_RMSE, 5)}, validation RMSE = {round(valid_RMSE, 5)}")
            
            #save best result
            if valid_RMSE < best_valid_RMSE:
                self.best_w = w
                self.best_bias = bias
                self.best_epoch = num_epoch
                best_valid_RMSE = valid_RMSE
                self.best_valid_RMSE = best_valid_RMSE

            #early stopping
            if valid_RMSE > best_valid_RMSE and num_epoch >= self.best_epoch + patience:
                self.stop_epoch = self.best_epoch + patience
                if verbose:
                    print("Early Stopping!")
                    print("="*10 + "validation result" + "="*10)
                    print(f"Best epoch is {self.best_epoch} with minimum validation error = {round(best_valid_RMSE, 5)}")
                return

        self.stop_epoch = num_epoch
        if verbose:
            print("="*10 + "Model result" + "="*10)
            print(f"Best epoch is {self.best_epoch} with minimum validation error = {round(best_valid_RMSE, 5)}")
                      
    def compute_valid_RMSE(self, X_valid, y_valid, w, bias):
        valid_loss = 0
        y_hat = np.dot(X_valid, w) + bias
        valid_loss = y_valid.reshape(-1, 1) - y_hat.reshape(-1, 1)
        valid_SSE = np.sum(np.square(valid_loss))
        valid_RMSE = np.sqrt(valid_SSE / X_valid.shape[0])
        return valid_RMSE

    def train_with_full_data(self, x, y, batch_size, epoch_size, learning_rate, lam_train, verbose = True):
        #Initialize parameter
        batch_size = batch_size 
        epoch_size = epoch_size 
        lr = learning_rate
        w, bias = self.initialize_params(x)

        #Adam optimizer
        lam = lam_train #L2 regularization
        beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1) #exponential decay rates for the moment estimates
        beta_2 = np.full(x[0].shape, 0.999).reshape(-1,1) #exponential decay rates for the moment estimates
        #m_t: first momentum vector, for momentum
        m_t = np.full(x[0].shape, 0).reshape(-1, 1)
        #v_t: second momemtum vector, for RMSprop
        v_t = np.full(x[0].shape, 0).reshape(-1, 1)
        m_t_b = 0.0
        v_t_b = 0.0
        t = 0
        epsilon = 1e-8

        for num_epoch in range(1, epoch_size + 1):
            #Shuffle when each epoch begin
            index = np.arange(x.shape[0])
            np.random.shuffle(index)
            x = x[index]
            y = y[index]

            SSE = 0
            for num_batch in range(int(x.shape[0]/batch_size)):
                t+=1
                x_batch = x[num_batch * batch_size:(num_batch + 1) * batch_size]
                y_batch = y[num_batch * batch_size:(num_batch + 1) * batch_size].reshape(-1,1)
                
                y_hat = np.dot(x_batch, w) + bias
                train_loss = y_batch - y_hat
                # 計算gradient
                g_t = np.dot(x_batch.transpose(), train_loss) * (-2) +  2 * lam * np.sum(w)
                g_t_b = train_loss.sum(axis=0) * (-2)
                m_t = beta_1*m_t + (1-beta_1)*g_t 
                v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
                m_cap = m_t/(1-(beta_1**t))
                v_cap = v_t/(1-(beta_2**t))
                m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
                v_t_b = 0.999*v_t_b + (1-0.999) * (g_t_b*g_t_b) 
                m_cap_b = m_t_b/(1-(0.9**t))
                v_cap_b = v_t_b/(1-(0.999**t))

                # 更新weight, bias
                w -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
                bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)
                SSE += np.sum(np.square(train_loss))

            # print(SSE)
            train_RMSE = np.sqrt(SSE / x.shape[0])
            self.train_loss.append(train_RMSE)
            if verbose:
                print(f"Epoch {num_epoch}, training RMSE = {round(train_RMSE, 5)}")

        self.best_w = w
        self.best_bias = bias

    def predict(self, X_test, w, bias):
        predict_value = np.dot(X_test, w) + bias
        return predict_value

In [ ]:
def write_to_csv(y_pred, file_name):
    with open(file_name, 'w', newline='') as csvf:
        # 建立 CSV 檔寫入器
        writer = csv.writer(csvf)
        writer.writerow(['Id','Predicted'])
        for i in range(int(y_pred.shape[0])):
            writer.writerow([i + 1, y_pred[i][0]])

# Preprocessing and model tuning

In [ ]:
DP = DataPreprocessor()
X_train, y_train = DP.preprocess_train_data(data_train)
X_test = DP.preprocess_test_data(data_test)

In [ ]:
#train_valid_ratio_list = [0.6, 0.7, 0.8, 0.9]
batch_size_list = [64, 128, 256, 512, 1024]
epoch_size_list = [20, 30, 50, 100]
learning_rate_list = [0.002, 0.005, 0.01, 0.025, 0.05]
lambda_list = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5]
np.random.seed(2021)

#Model saving
global_best_w = 0
global_best_bias = 0
global_best_RMSE = 1000000

#Hyperparameter saving
best_train_valid_ratio = 0
best_batch_size = 0
best_epoch_size = 0
best_learning_rate = 0
best_lambda = 0

# for train_valid_ratio in train_valid_ratio_list:
for batch_size in batch_size_list:
    for epoch_size in epoch_size_list:
        for learning_rate in learning_rate_list:
            for lam in lambda_list:
                print("=" * 10,"Start training model", "="*10)
                print(f"batch size = {batch_size}, epoch_size = {epoch_size}, learning rate = {learning_rate}, lambda = {lam}")
                model = LinearRegression()
                model.train(X_train, y_train, batch_size, epoch_size, learning_rate, lam, verbose = True)
                if model.best_valid_RMSE < global_best_RMSE:
                    global_best_RMSE = model.best_valid_RMSE
                    global_best_w = model.best_w
                    global_best_bias = model.best_bias
                    best_batch_size = batch_size
                    best_epoch_size = epoch_size
                    best_stop_epoch_size = model.best_epoch
                    best_learning_rate = learning_rate
                    best_lambda = lam

========== Start training model ==========
batch size = 64, epoch_size = 20, learning rate = 0.002, lambda = 0.001
Epoch 1, training RMSE = 28.45004, validation RMSE = 26.97757
Epoch 2, training RMSE = 26.14901, validation RMSE = 25.24989
Epoch 3, training RMSE = 24.95117, validation RMSE = 24.35827
Epoch 4, training RMSE = 24.14988, validation RMSE = 23.81528
Epoch 5, training RMSE = 23.58159, validation RMSE = 23.34367
Epoch 6, training RMSE = 23.13333, validation RMSE = 22.86589
Epoch 7, training RMSE = 22.66811, validation RMSE = 22.54766
Epoch 8, training RMSE = 22.29012, validation RMSE = 22.13532
Epoch 9, training RMSE = 21.89954, validation RMSE = 21.78404
Epoch 10, training RMSE = 21.54923, validation RMSE = 21.37929
Epoch 11, training RMSE = 21.18159, validation RMSE = 21.02205
Epoch 12, training RMSE = 20.83371, validation RMSE = 20.63436
Epoch 13, training RMSE = 20.46816, validation RMSE = 20.30677
Epoch 14, training RMSE = 20.11563, validation RMSE = 19.9609
Epoch 15, tra

In [ ]:
print("=" * 10, " Best Model result ", "=" * 10)
print(f"Batch Size = {best_batch_size}, Epoch Size = {best_epoch_size} (Actually running {best_stop_epoch_size} epoch), Learning rate = {best_learning_rate}")
print(f"Best lambda = {best_lambda}")
print(f"Validation RMSE = {global_best_RMSE}")

==========  Best Model result  ==========
Batch Size = 128, Epoch Size = 100 (Actually running 93 epoch), Learning rate = 0.005
Best lambda = 0.001
Validation RMSE = 0.6552837787040128


In [ ]:
train_all_model = LinearRegression()
train_all_model.train_with_full_data(X_train, y_train, best_batch_size, best_stop_epoch_size, best_learning_rate, best_lambda,verbose = False)
final_w = train_all_model.best_w
final_bias = train_all_model.best_bias

In [ ]:
#Training with all data finally
predict_model = LinearRegression()
y_pred = predict_model.predict(X_test, final_w, final_bias)

In [ ]:
file_name = 'prediction.csv'
write_to_csv(y_pred, file_name)